# Convert and test Matthew's python exposure code


In [1]:
%matplotlib inline
from importlib import reload
import glob
from astropy.io import fits
import exposure as exp; reload(exp)

<module 'exposure' from '/nfs/farm/g/glast/u/burnett/analysis/agn_timing/code/exposure.py'>

### Locate files used for catalog
Note that the data files are not FT1 but binned: code here uses only for GTI data

In [2]:
os.environ['FERMI']='/nfs/farm/g/glast/g/catalog/pointlike/fermi'

yearly_ft2files = sorted(glob.glob('/nfs/farm/g/glast/g/catalog/P8_P305/ft2_20*.fits'))
yearly_datafiles= sorted(glob.glob(os.path.expandvars('$FERMI/data/P8_P305/yearly/*.fits')))
ft2= yearly_ft2files[0]
ft1= yearly_datafiles[0]

### Create an Exposure object wtih the first year

In [3]:
ef = exp.Exposure(ft2, ft1)

Effective area setep {}
Processing GTI...
Processed 5497 intervals with 25801634 s live time
Loading FT2 files /nfs/farm/g/glast/g/catalog/P8_P305/ft2_2008.fits
Processing the FT2 file (calculating overlap with GTI)...


In [4]:
ef((0,0), [100,1000.],)

Eff call: ((array([ 100., 1000.]), 0.2125))
Eff call: ((array([ 100., 1000.]), 0.2375))
Eff call: ((array([ 100., 1000.]), 0.2625))
Eff call: ((array([ 100., 1000.]), 0.28750002))
Eff call: ((array([ 100., 1000.]), 0.3125))
Eff call: ((array([ 100., 1000.]), 0.33749998))
Eff call: ((array([ 100., 1000.]), 0.3625))
Eff call: ((array([ 100., 1000.]), 0.3875))
Eff call: ((array([ 100., 1000.]), 0.41250002))
Eff call: ((array([ 100., 1000.]), 0.4375))
Eff call: ((array([ 100., 1000.]), 0.46249998))
Eff call: ((array([ 100., 1000.]), 0.4875))
Eff call: ((array([ 100., 1000.]), 0.5125))
Eff call: ((array([ 100., 1000.]), 0.5375))
Eff call: ((array([ 100., 1000.]), 0.5625))
Eff call: ((array([ 100., 1000.]), 0.5875))
Eff call: ((array([ 100., 1000.]), 0.6125))
Eff call: ((array([ 100., 1000.]), 0.6375))
Eff call: ((array([ 100., 1000.]), 0.6625))
Eff call: ((array([ 100., 1000.]), 0.6875))
Eff call: ((array([ 100., 1000.]), 0.7125))
Eff call: ((array([ 100., 1000.]), 0.7375))
Eff call: ((arra

array([[2.95318287e+09, 9.05983367e+09],
       [2.34436485e+09, 7.96316743e+09]])